# Popular Items from Yelp reviews
###### Ashwin Sura Ravi | Dileep Kumar Gunda | Suryakiran Suresh Gurumoorthy Iyer

In [ ]:
# Install following packages!

import sys
!{sys.executable} -m pip install nltk
!{sys.executable} -m pip install inflect

## Step-1: Gather Data
#### Data used in the code is located at, https://drive.google.com/open?id=1C4laG6BFKFyiPjy27dRyBu6bG-uUzgBT

### Filter Yelp Dataset

Filter Yelp Dataset for restaurants(category) with number of reviews between 100 and 150. <br> <br>
*INPUT:* <br>
**yelp_academic_dataset_business.json**: Meta Information(name, ratings, #reviews, etc) about all business in Yelp including restaurants <br>
**yelp_academic_dataset_review.json**: Contains reviews corresponding to all businesses present in above file<br><br>
*OUTPUT:* <br>
**restaurants.json**: list of restaurants sayisfying given condition <br>
**reviews.json**: contains reviews corresponding to all restaurants in restaurants.json

In [3]:
import json
import re
import sys

review_count_threshold = 100
restaurant_ids = []
restaurant_file = 'data/restaurants.json'
review_file = 'data/reviews.json'

In [5]:
# extract restaurants

restaurant_writer = open(restaurant_file, 'w')
total_reviews = 0
restaurant_ids = []
with open('data/yelp_academic_dataset_business.json') as json_file:
    for line in json_file:
        json_line = json.loads(line)
        #print "business: %s, review count: %s" % (json_line['name'], json_line['review_count'])
        categories = json_line['categories']
        if categories != None:
            categories = categories.lower()
        if json_line['review_count'] > review_count_threshold and json_line['review_count'] < 150 and 'restaurant' in categories:
            restaurant_ids.append(json_line['business_id'])
            total_reviews += json_line['review_count']
            restuarant = {}
            restuarant['business_id'] = json_line['business_id']
            restuarant['name'] = json_line['name']
            restuarant['stars'] = json_line['stars']
            restaurant_writer.write('%s\n' % (json.dumps(restuarant)))

restaurant_writer.close()
print 'total restaurants: %s, total reviews: %s, avg reviews: %d' % (len(restaurant_ids), total_reviews, total_reviews/len(restaurant_ids))

total restaurants: 3338, total reviews: 408196, avg reviews: 122


In [8]:
# extract reviews for restaurants

review_writer = open(review_file, 'w')
total_reviews = 0
with open('data/yelp_academic_dataset_review.json') as json_file:
    for line in json_file:
        json_line = json.loads(line)
        if json_line['business_id'] in restaurant_ids:
            del json_line['user_id']
            del json_line['funny']
            del json_line['cool']
            review_writer.write('%s\n' % (json.dumps(json_line)))
            total_reviews += 1

review_writer.close()
print 'total reviews written, %s' % (total_reviews)

total reviews written, 618060


## Step-2: Generating Labels and Truth values (Training and Test set)

Computing popular items, require identifying food entities in each review and sentiment of each entity. Yelp dataset doesn't contain such data nor the menu for each restaurant. So below we selected 7 restaurants and annotated every review for these 7 restaurants with food entities and the user sentiment for each entity. <br>
Below are the 7 restaurants names, there exists 7 JSON files with same name, each containing reviews for that corresponding restaurant. <br><br>
9UTpmQ4OhX5jNFUIu7dPPQ <br>
dusNIzdCaH6EoLl2hRy6cQ <br>
FDEm-c3NAXnTVtl-hgzAhA <br>
Hdzo5ggPswyv-8ZlW0PVLw <br>
ILA41PhErg4bl1royzDX4g <br>
L0aSDVHNXCl6sY4cfZQ-5Q <br>
RGK23CEkDfYHWtUbRhA-bQ <br>

In [ ]:
# Selecting 7 restaurants and extracting reviews of them
import json

restaurant_file = 'data/restaurants.json'
selected_restaurants = []
with open(restaurant_file, 'r') as json_file:
    for line in json_file:
        json_line = json.loads(line)
        if not json_line['business_id'].startswith('-'):
            selected_restaurants.append(json_line['business_id'])
            if len(selected_restaurants) == 7:
                break

# print(selected_restaurants)

# extracting reviews for those selected restaurants
review_file = 'data/reviews.json'
for res_id in selected_restaurants:
    res_writer = open('data/tmp/'+res_id + '.json', 'w')
    with open(review_file, 'r') as json_file:
        for line in json_file:
            json_line = json.loads(line)
            if json_line['business_id'] == res_id:
                res_writer.write('%s\n' % (json.dumps(json_line)))
    res_writer.close()

print('Done.')

### Annotation Tool
Tool to annotate each of the above files with entity and its corresponding sentiment. It writer annotation to file names **tag_{restaurant-id}.json**. Later all these files, reviews and annotations are merged into single files named **reviews.json** and **review_tags.json**. <br><br>
*Hereafter reviews.json file corresponds to reviews of the above selected 7 restaurants.They are inside **tmp** folder*

In [ ]:
ptr = 0

In [27]:
import json
import sys

review_file = 'data/tmp/h_ypoQ2rmwX8UGSmohsGiQ.json'
annotate_file = 'data/tmp/tag_h_ypoQ2rmwX8UGSmohsGiQ.json'
annotate_writer = open(annotate_file, 'a+')
with open(review_file, 'r') as json_file:
    i = 0
    for line in json_file:
        if i >= ptr:
            json_line = json.loads(line)
            annotation = {}
            annotation['review_id'] = json_line['review_id']
            annotation['entities'] = []
            try:
                print('\nREVIEW: #%d\n%s\n' % (i, json_line['text']))

                print('next entity: (<entity_name>/n)')
                entity = raw_input()
                while entity != 'n':
                    print('sentiment: ')
                    sentiment = int(raw_input())
                    annotation['entities'].append({'entity': entity, 'sentiment': sentiment})
                    print('next entity: (<entity_name>/n)')
                    entity = raw_input()
                annotate_writer.write('%s\n' % (json.dumps(annotation)))
                i = i+1
                print('continue to next review: (y/n)')
                y = raw_input()
                if y == 'n':
                    ptr = i
                    annotate_writer.close()
                    print("continue next time from record number: %s" % (ptr))
                    break
            except ValueError as e:
                annotate_writer.close()
                ptr = i
                print(e)
                sys.exit(1)
        else:
            i = i+1
annotate_writer.close()
print('Done.')


REVIEW: #21
This place is an oasis in the busy FCP food court.they have their own enclosed seating area. They have excellent quiches, salads and baked goods. Try their chocolate chip cookies. Full service espresso bar and patio in the summer.

next entity: (<entity_name>/n)
quiches
sentiment: 
1
next entity: (<entity_name>/n)
salads
sentiment: 
1
next entity: (<entity_name>/n)
chocolate chip cookies
sentiment: 
1
next entity: (<entity_name>/n)
n
continue to next review: (y/n)
y

REVIEW: #22
Honestly, wow. I love this place - the decor, food, coffee, friendly staff all come together beautifully. What an oasis in the financial district.

Absolutely try their baked goods. 

Maman serves up daily fruit tarts (usually later in the afternoon) beautifully made with flavours melding together perfectly. They're also not too sweet, have lots of fresh fruit and the presentation's on-point (peach blueberry almond tart, apple tart)! Usually the pound cakes are served up as samples. I prefer the or

next entity: (<entity_name>/n)
chocolate croissants
sentiment: 
-1
next entity: (<entity_name>/n)
n
continue to next review: (y/n)
y

REVIEW: #31
This Parisian style bakery/cafe place is super cute. It reminds me of a little cafe you can find in the southern region of France. This is the second location of the Maman chain. The first cafe is in the SoHo neighbourhood in Manhattan. The cafe has a rustic feel to it and it also feels very cozy. Like other Yelpers say, this cafe is actually difficult to find. It's in the upstairs food court in First Canadian Place near the IQ restaurant. Be sure to come earlier during lunch to avoid the lunch rush. The line up gets hectic around here since it is located in the financial district. 

Went during lunch and my party only ordered the "two salads for $11" option. Okay hands down, their salads are pretty good especially the one that has cranberries (I believe the salad was called Julie). Though the salads are pricey, I think it is worth trying it 

## Step-3: Extracting (food) Entities from reviews
### Preparing Generic Menu of restaurants
We need restaurants menu to extract food entities from reviews. We couldnt find exact menu, but found yelp older dataset which contains menu of over 1000 restaurants. Menu is captured in file **yelp-old-data-restaurants.json**. Only menu is extracted into file **yelp-menu.txt** and its processed to refine the menu like removing special characters, splitting multiple items into individual lines. The processed and refined menu is captured in **proc_menu.txt**. <br><br>
Over the time, *proc_menu.txt* has been refined directly using regular expressions in the file-editor. *Re-running the block will override the proc_menu.txt and the changes will be lost.*

In [4]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dkgun\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\dkgun\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dkgun\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\dkgun\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [79]:
tokens = nltk.word_tokenize("I had grilled chicken and highly recommend it")
print nltk.pos_tag(tokens)
tokens = nltk.word_tokenize("I had beef pork pizza")
print nltk.pos_tag(tokens)

[('I', 'PRP'), ('had', 'VBD'), ('grilled', 'VBN'), ('chicken', 'NNS'), ('and', 'CC'), ('highly', 'RB'), ('recommend', 'VB'), ('it', 'PRP')]
[('I', 'PRP'), ('had', 'VBD'), ('beef', 'NN'), ('pork', 'NN'), ('pizza', 'NN')]


In [4]:
# prepare old yelp menu

# unwanted_chars_re = '[\'\"\-0-9\\\\]'
# dish_file = 'data/yelp-old-data-restaurants.json'
# dish_writer = open('data/yelp-menu.txt', 'w')
# dish_names = []
# with open(dish_file) as json_file:
#     for line in json_file:
#         json_line = json.loads(line)
#         if len(json_line['items']) > 0:
#             for item in json_line['items']:
#                 dish_names.append(re.sub(unwanted_chars_re, '', item['name'].lower()))
#                 dish_writer.write(re.sub(unwanted_chars_re, '', item['name'].lower())+'\n')

# dish_writer.close()
# print 'Total Dish names: %d' % (len(dish_names))

In [3]:
# process yelp menu

# import re

# yelp_menu_file = 'data/yelp-menu.txt'
# processed_menu_writer = open('data/proc_menu1.txt', 'w')
# yelp_menu = []
# unwanted_chars_re = '[\'\"\-0-9]'
# with open(yelp_menu_file, 'r') as txt_file:
#     for line in txt_file:
#         items = line.split('/')
#         for i in items:
#             i = re.sub(unwanted_chars_re, '', i.lower().strip())
#             i = re.sub('[\,\(\)\+]', ' ', i)
#             yelp_menu.append(i)
#             processed_menu_writer.write(i+'\n')
# processed_menu_writer.close()
# print('Done.')

Done.


### Extracting entities from reviews
Extracting food entities for each review in reviews.json file. Its a general observation that any menu has a noun in it like *chicken burger*. The code uses this fact to detect nouns in each review, considers moving window, upto length 4 around that noun and then checks if that phrase is present in processed yelp menu. All such detected food entities are written to file below. <br><br>
*OUTPUT*<br>
**predicted_tags.json**: contains model extracted (food) entities for each review 

In [ ]:
# helper funtions to transform and compare two menu items
from nltk.stem import PorterStemmer
import inflect

inf = inflect.engine()
porter = PorterStemmer()

def normalize_word(word):
    return word
#     return porter.stem(word)

def compare_words(word1, word2):
    word1_pl = inf.plural(word1)
    word2_pl = inf.plural(word2)
    if word1 == word2 or word1 == word2_pl or word1_pl == word2:
        return True
    return False

In [9]:
# Helper functions to search given phrase in processed yelp menu
import nltk
import re

# read processed yelp menu
# for now reading from variable yelp_menu
def read_yelp_menu():
    yelp_menu = []
    with open('data/proc_menu.txt', 'r') as txt_file:
        for line in txt_file:
            yelp_menu.append(line.lower().strip())
    return yelp_menu
    
def search_menu(item, yelp_menu):
    # discard single character item search
    if len(item) == 1:
        return False
    unwanted_chars_re = '[\'\"\-0-9]'
    item = re.sub(unwanted_chars_re, '', item.lower().strip())
    item = item.split()
    if len(item) == 1:
        # search only pefix of yelp menu
        for yelp_item in yelp_menu:
            if len(yelp_item.strip().split()) > 0:
                # first word of yelp_item
                y_word = normalize_word(yelp_item.strip().split()[0])
                i_word = normalize_word(item[0])
                if y_word == i_word:
                    return True
    elif len(item) > 1:
        # search if all items are present in single menu instance of yelp menu
        match = False
        for yelp_item in yelp_menu:
            match = True
            for w in item:
                w = normalize_word(w)
                if match:
                    match = False
                    for y in yelp_item.split():
                        y = normalize_word(y)
                        if y == w:
                            match = True
            if match:
                return True
        return match
    return False

def segment_review(review, yelp_menu):
    review_tokens = nltk.word_tokenize(review)
    token_pos = nltk.pos_tag(review_tokens)
    i = 0
    items = set()
    while i < len(review_tokens):
        if token_pos[i][1].startswith("NN"):
            print(review_tokens[i]+":")
            included = False
            # previous word
            if i-1 >= 0:
                item = " ".join(review_tokens[i-1:i+1])
                if token_pos[i-1][1].startswith("VB") or token_pos[i-1][1].startswith("JJ"):
                    if search_menu(item, yelp_menu):
                        items.add(item)
                        included = True
            # next word     
            if i+1 < len(review_tokens) and not included:
                item = " ".join(review_tokens[i:i+2])
                if token_pos[i+1][1].startswith("VB") or token_pos[i+1][1].startswith("JJ"):
                    if search_menu(item, yelp_menu):
                        items.add(item)
                        included = True
            # only current word
        i = i+1
    print 'items: %s' % (items)
    
def segment_review_len(review, yelp_menu):
    review_tokens = nltk.word_tokenize(review)
    token_pos = nltk.pos_tag(review_tokens)
    i=0
    items = set()
    included = []
    while i < len(review_tokens):
        if token_pos[i][1].startswith("NN"):
            found = False
            for l in range(4, 0, -1):
                if not found:
                    for x in range(i-l+1, i+1):
                        y = x+l
                        if x >= 0 and y <= len(review_tokens):
                            item = " ".join(review_tokens[x:y])
                            if len(included)==0 or included[-1][1]<x:
                                if token_pos[x][1].startswith("VB") or token_pos[x][1].startswith("JJ") or token_pos[x][1].startswith("RB") or token_pos[x][1].startswith("NN"):
                                    if search_menu(item, yelp_menu):
                                        if len(included)>0 and x == 1+included[-1][1]:
                                            included[-1] = (included[-1][0], y-1)
                                        else:
                                            included.append((x,y-1))
                                        found=True
                                        break
        i+=1
    for x,y in included:
        items.add(" ".join(review_tokens[x:y+1]))
    return items
                    
                

In [10]:
print search_menu('death by chocolate', read_yelp_menu())

True


In [44]:
# Helper functions to compute Recall and Precision given Predicted and True values
import nltk
import re

def remove_stop_words(words):
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(words)
    filered_words = [w for w in word_tokens if not w in stop_words]
    return " ".join(filered_words)

def preprocess(words):
    words = remove_stop_words(words)
    words = words.split()
    out = []
    for i in range(len(words)):
        word = words[i]
        word = word.lower()
        word = re.sub('[^A-Za-z0-9]+', '', word)
        p = nltk.PorterStemmer()
        word = p.stem(word)
        out.append(word)
    out.sort()
    out = ' '.join(out)
    return out

def custom_recall(true_y, pred_y):
    true_y = list(set(true_y))
    pred_y = list(set(pred_y))
    if len(true_y) == 0:
        return 0.0, 0.0
    if len(pred_y) == 0:
        return 0.0, 0.0
    for i in range(len(true_y)):
        curr = preprocess(true_y[i])
        true_y[i] = curr
    for i in range(len(pred_y)):
        curr = preprocess(pred_y[i])
        pred_y[i] = curr

#     print('true_y: %s' % (true_y))
#     print('pred_y: %s' % (pred_y))
    # compute recall - fraction of true values predicted
    count_arr = [0] * len(true_y)
    for i in range(len(true_y)):
        for j in range(len(pred_y)):
          if true_y[i] in pred_y[j] or pred_y[j] in true_y[i]:
            count_arr[i] = 1
    recall = sum(count_arr) * 1.0 / len(true_y)
    
    # compute precision -  fraction of predicted values that are true
    count_arr = [0] * len(pred_y)
    for j in range(len(pred_y)):
        for i in range(len(true_y)):
          if true_y[i] in pred_y[j] or pred_y[j] in true_y[i]:
            count_arr[j] = 1
    precision = sum(count_arr) * 1.0 / len(pred_y)
    return recall, precision

#####  Main block that extracts food entities. Takes around 6 hrs to run.

In [48]:
# read reviews and extract food entities
import json
import re

# read true value entities for each review
# uncomment below to run!

# review_tags = {}
# with open('data/review_tags.json', 'r') as json_file:
#     for line in json_file:
#         json_line = json.loads(line)
#         review_tags[json_line['review_id']] = json_line['entities']


# print 'starting'
# pred_writer = open('data/predicted_tags.json', 'w')
# unwanted_chars_re = '[\'\"\-0-9\\\\]'
# ptr = 0
# with open('data/reviews.json', 'r') as json_file:
#     lines_read = 0
#     yelp_menu = read_yelp_menu()
#     avg_recall = 0.0
#     avg_precision = 0.0
#     for line in json_file:
#         if lines_read >= ptr:
#             json_line = json.loads(line)
#             review =  re.sub(unwanted_chars_re, '', json_line['text'].lower())
#             review = review.replace('\n', ' ')
#             print('Review: #%d' % (lines_read+1))
# #             print '\nreview: %s' % (review)
#             pred_items = segment_review_len(review, yelp_menu)
#             true_items = [ent['entity'] for ent in review_tags[json_line['review_id']]]
# #             print 'pred_items: %s' % (pred_items)
# #             print 'true_items: %s' % (true_items)
#             recall, precision = custom_recall(true_items, pred_items)
# #             print('Recall: %f Precision: %f' % (recall, precision))
#             avg_recall += recall
#             avg_precision += precision
#             pred_value = {}
#             pred_value['review_id'] = json_line['review_id']
#             pred_value['entities'] = list(pred_items)
#             pred_writer.write('%s\n' % (json.dumps(pred_value)))
# #             y = raw_input("Continue to next review: (y/n)")
# #             if y == 'n':
# #                 ptr = lines_read
# #                 break
#         lines_read = lines_read +1
# pred_writer.close()
# print 'Avg Recal: %f Avg Precision: %f' % (avg_recall, avg_precision)

starting
Review: #1
Review: #2
Review: #3
Review: #4
Review: #5
Review: #6
Review: #7
Review: #8
Review: #9
Review: #10
Review: #11
Review: #12
Review: #13
Review: #14
Review: #15
Review: #16
Review: #17
Review: #18
Review: #19
Review: #20
Review: #21
Review: #22
Review: #23
Review: #24
Review: #25
Review: #26
Review: #27
Review: #28
Review: #29
Review: #30
Review: #31
Review: #32
Review: #33
Review: #34
Review: #35
Review: #36
Review: #37
Review: #38
Review: #39
Review: #40
Review: #41
Review: #42
Review: #43
Review: #44
Review: #45
Review: #46
Review: #47
Review: #48
Review: #49
Review: #50
Review: #51
Review: #52
Review: #53
Review: #54
Review: #55
Review: #56
Review: #57
Review: #58
Review: #59
Review: #60
Review: #61
Review: #62
Review: #63
Review: #64
Review: #65
Review: #66
Review: #67
Review: #68
Review: #69
Review: #70
Review: #71
Review: #72
Review: #73
Review: #74
Review: #75
Review: #76
Review: #77
Review: #78
Review: #79
Review: #80
Review: #81
Review: #82
Review: #83
Revi

Review: #639
Review: #640
Review: #641
Review: #642
Review: #643
Review: #644
Review: #645
Review: #646
Review: #647
Review: #648
Review: #649
Review: #650
Review: #651
Review: #652
Review: #653
Review: #654
Review: #655
Review: #656
Review: #657
Review: #658
Review: #659
Review: #660
Review: #661
Review: #662
Review: #663
Review: #664
Review: #665
Review: #666
Review: #667
Review: #668
Review: #669
Review: #670
Review: #671
Review: #672
Review: #673
Review: #674
Review: #675
Review: #676
Review: #677
Review: #678
Review: #679
Review: #680
Review: #681
Review: #682
Review: #683
Review: #684
Review: #685
Review: #686
Review: #687
Review: #688
Review: #689
Review: #690
Review: #691
Review: #692
Review: #693
Review: #694
Review: #695
Review: #696
Review: #697
Review: #698
Review: #699
Review: #700
Review: #701
Review: #702
Review: #703
Review: #704
Review: #705
Review: #706
Review: #707
Review: #708
Review: #709
Review: #710
Review: #711
Review: #712
Review: #713
Review: #714
Review: #715

### Filtering extracted entities
Above generated entities have heavy noise. This is carry forwarded from yelp menu and also the way phrase matching is implemented in the above blocks. To reduce this noise, we pass these extracted entities to nltk food corpus, [https://wordnet.princeton.edu/documentation/lexnames5wn]. This identifies noise data and is captured in file **non-entities.txt**. Later these non-entities are removed from extracted entities and written into **filtered_predicted_tags.json**

In [33]:
# helper functions to merge, sort, compare two menu items like 'chicken salad' and 'salad with chicken'
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

# output: {"review_id": {"business_id": "", "stars": #}}
def get_review_restaurant_map():
    rev_res_map = {}
    with open('data/reviews.json', 'r') as json_file:
        for line in json_file:
            json_line = json.loads(line)
            review_info = {}
            review_info['business_id'] = json_line['business_id']
            review_info['stars'] = json_line['stars']
            rev_res_map[json_line['review_id']] = review_info
    return rev_res_map

def get_true_entities(rev_res_map):
    item_scores_true = {}
    with open('data/review_tags.json', 'r') as json_file:
        for line in json_file:
            item_scores = {}
            json_line = json.loads(line)
            review_id = json_line['review_id']
            business_id = rev_res_map[review_id]['business_id']
            if business_id in item_scores_true:
              item_scores = item_scores_true[business_id]
            for entity in json_line['entities']:
              food_item = entity['entity'].lower()
              if food_item not in item_scores:
                item_scores[food_item] = 0
              item_scores[food_item] += entity['sentiment']
            item_scores_true[business_id] = item_scores
        for res in item_scores_true:
          item_scores_true[res] = sorted(item_scores_true[res].items(), key=lambda x: x[1], reverse=True)
    return item_scores_true

def sort_entities_map(ent_map):
    for res in ent_map:
          ent_map[res].sort(key=lambda x: x[1], reverse=True)
    return ent_map

# map{res_id => [entities]}
def merge_entities(res_ent_map):
    for res_id in res_ent_map:
        res_ent_map[res_id] = merge_similar_items(res_ent_map[res_id])
    return res_ent_map

# items is list of tuple like ('item_name', #occurences)
def merge_similar_items(items):
    if len(items) == 0:
        return items
    i=0
    while i < len(items):
        j = i+1
        while j < len(items):
            if is_similar(items[i][0], items[j][0]):
                # print('SIMILAR: %s <-> %s' % (items[i][0], items[j][0]))
                items[i] = (items[i][0], items[i][1]+items[j][1])
                del items[j]
            else:
                j +=1
        i += 1
    return items

# remove filter words
# stem each word
def is_similar(item1, item2):
    item1 = remove_stop_words(item1)
    item1 = preprocess(item1)
    item2 = remove_stop_words(item2)
    item2 = preprocess(item2)
    
    # its a mismatch if items of different length
    if len(set(item1.split())) != len(set(item2.split())):
        return False
    
    found = True
    for i1 in item1.split():
        found = False
        for i2 in item2.split():
            if i1 == i2:
                found = True
        if not found:
            return False
    return True
    
def remove_stop_words(words):
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(words)
    filered_words = [w for w in word_tokens if not w in stop_words]
    return " ".join(filered_words)

##### Preparing Custom non-entities

In [50]:
# get count of predicted entities
def get_pred_entities(rev_res_map, pred_file):
    item_scores_pred = {}
    with open(pred_file, 'r') as json_file:
        for line in json_file:
            item_scores = {}
            json_line = json.loads(line)
            review_id = json_line['review_id']
            business_id = rev_res_map[review_id]['business_id']
            if business_id in item_scores_pred:
              item_scores = item_scores_pred[business_id]
            for entity in json_line['entities']:
              food_item = entity.lower()
              if food_item not in item_scores:
                item_scores[food_item] = 0
              item_scores[food_item] += 1
            item_scores_pred[business_id] = item_scores
        for res in item_scores_pred:
          item_scores_pred[res] = sorted(item_scores_pred[res].items(), key=lambda x: x[1], reverse=True)
    return item_scores_pred

In [156]:
rev_res_map = get_review_restaurant_map()

pred_entities = get_pred_entities(rev_res_map, 'data/predicted_tags.json')
pred_entities = merge_entities(pred_entities)

true_entities = get_true_entities(rev_res_map)
true_entities = merge_entities(true_entities)

for res_id in pred_entities:
    print '\nres_id: %s' % (res_id)
    print 'Pred: %s\n' % (pred_entities[res_id][:20])
    print 'True: %s\n' % (true_entities[res_id][:20])


res_id: Hdzo5ggPswyv-8ZlW0PVLw
Pred: [(u'gelato', 99), (u'flavor', 27), (u'ice cream', 23), (u'chocolate', 21), (u'selection', 23), (u'cup', 21), (u'size', 12), (u'way', 11), (u'fruit', 11), (u'dinner', 9), (u'mango', 9), (u'day', 9), (u'pistachio', 8), (u'fruity', 8), (u'coffee', 7), (u'home', 7), (u'chocolate chili', 10), (u'peanut butter', 7), (u'kids', 7), (u'family', 6)]

True: [(u'ferrero rocher', 8), (u'peanut butter', 9), (u'mango', 8), (u'pistachio', 7), (u'strawberry cheesecake', 6), (u'chocolate', 6), (u'grapefruit', 5), (u'grasshopper', 5), (u'lemon', 5), (u'chocolate chili', 5), (u'raspberry', 4), (u'banana', 4), (u'strawberry', 4), (u'coconut', 4), (u'gelato', 4), (u'stracciatella', 3), (u'double fudge chocolate brownie', 3), (u'death by chocolate', 3), (u'sunshine', 3), (u'pear', 3)]


res_id: dusNIzdCaH6EoLl2hRy6cQ
Pred: [(u'chipotle', 51), (u'strip', 28), (u'burrito', 28), (u'line', 18), (u'rice', 17), (u'meat', 13), (u'chicken', 12), (u'express', 11), (u'bowl', 11), 

In [14]:
# preparing custom list of non-entities
from nltk.corpus import wordnet

# return True is atleast on word in words[]] belongs to Food.
def is_food(words):
    words = words.lower().split()
    for word in words:
        syns = wordnet.synsets(str(word), pos = wordnet.NOUN)
        for syn in syns:
            if 'food' in syn.lexname():
                return True
    return False

def write_custom_stop_entities(pred_entities_map, true_entities_map):
    stop_entity_writer = open('data/non_entities.txt', 'w')
    for res_id in pred_entities_map:
        for pred_entity,_ in pred_entities_map[res_id]:
            similar = False
            for true_entity,_ in true_entities_map[res_id]:
                if not similar and is_similar(pred_entity, true_entity):
                    similar = True
            if not similar and not is_food(pred_entity):
                stop_entity_writer.write('%s\n' % (pred_entity))
    stop_entity_writer.close()

# uncomment this to write non_entites.txt
# write_custom_stop_entities(pred_entities, true_entities)

In [46]:
# helper functions to remove stop-words, non-entities from extracted entities

from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
import inflect

inf = inflect.engine()

def get_non_entities():
    non_entities = set()
    with open('data/non_entities.txt', 'r') as txt_file:
        for line in txt_file:
            non_entities.add(line.lower().strip())
    return non_entities

def remove_trailing_stop_words(words):
    words = words.split()
    length = len(words)
    len_changed = True
    while len_changed and length > 0:
        len_changed = False
        w = remove_stop_words(words[-1])
        if w == "":
            len_changed = True
            length -= 1
            del words[-1]
    return " ".join(words)

def remove_non_entities(entities, non_entities):
    i = 0
    while i < len(entities):
        # entities is list of 'names'
        entities[i] = remove_trailing_stop_words(entities[i])
        if not entities[i] or entities[i] == "":
            del entities[i]
        elif entities[i] in non_entities or inf.plural(entities[i]) in non_entities:
            del entities[i]
        else:
            i += 1
    return entities

def remove_non_entities_pred(pred_entities_map, non_entities):
    for res_id in pred_entities_map:
        entities = pred_entities_map[res_id]
        i = 0
        while i < len(entities):
            # entities is tuple ('name', #score)
            if entities[i][0] in non_entities or inf.plural(entities[i][0]) in non_entities:
                del entities[i]
            else:
                i += 1
        pred_entities_map[res_id] = entities
    return pred_entities_map

In [182]:
non_entities = list(get_non_entities())
pred_entities = remove_stop_entities(pred_entities, stop_entities)

for res_id in pred_entities:
    print '\nres_id: %s' % (res_id)
    print 'Pred: %s\n' % (pred_entities[res_id][:20])
    print 'True: %s\n' % (true_entities[res_id][:20])


res_id: Hdzo5ggPswyv-8ZlW0PVLw
Pred: [(u'gelato', 99), (u'ice cream', 23), (u'chocolate', 21), (u'mango', 9), (u'pistachio', 8), (u'fruity', 8), (u'coffee', 7), (u'chocolate chili', 10), (u'peanut butter', 7), (u'grapefruit', 5), (u'tiramisu', 5), (u'cake', 6), (u'banana', 5), (u'strawberry', 7), (u'mint', 5), (u'cream', 5), (u'cheesecake', 5), (u'guava', 4), (u'caramel', 4), (u'sorbets', 6)]

True: [(u'ferrero rocher', 8), (u'peanut butter', 9), (u'mango', 8), (u'pistachio', 7), (u'strawberry cheesecake', 6), (u'chocolate', 6), (u'grapefruit', 5), (u'grasshopper', 5), (u'lemon', 5), (u'chocolate chili', 5), (u'raspberry', 4), (u'banana', 4), (u'strawberry', 4), (u'coconut', 4), (u'gelato', 4), (u'stracciatella', 3), (u'double fudge chocolate brownie', 3), (u'death by chocolate', 3), (u'sunshine', 3), (u'pear', 3)]


res_id: dusNIzdCaH6EoLl2hRy6cQ
Pred: [(u'chipotle', 51), (u'burrito', 28), (u'rice', 17), (u'meat', 13), (u'chicken', 12), (u'beans', 10), (u'salsa', 11), (u'meal', 8), (

##### write filtered predicted(extracted) entities

In [47]:
import json

def filter_predicted_entities(non_entities):
    filter_pred_tag_writer = open('data/filtered_predicted_tags.json', 'w')
    with open('data/predicted_tags.json', 'r') as json_file:
        for line in json_file:
            json_line = json.loads(line)
            json_line['entities'] = remove_non_entities(json_line['entities'], non_entities)
            filter_pred_tag_writer.write('%s\n' % (json.dumps(json_line)))
    filter_pred_tag_writer.close()
    
# uncomment to write filtered predicted tags
# filter_predicted_entities(list(get_non_entities()))        

## Step-4: Compute Popular Items

### Model 1: based #occurences of each extracted entity in whole review set

In [51]:
import json
import re

rev_res_map = get_review_restaurant_map()

pred_entities = get_pred_entities(rev_res_map, 'data/filtered_predicted_tags.json')
pred_entities = merge_entities(pred_entities)
pred_entities = sort_entities_map(pred_entities)

true_entities = get_true_entities(rev_res_map)
true_entities = merge_entities(true_entities)
true_entities = sort_entities_map(true_entities)

for res_id in pred_entities:
    print '\nres_id: %s' % (res_id)
    print 'Pred: %s\n' % (pred_entities[res_id][:10])
    print 'True: %s\n' % (true_entities[res_id][:10])


res_id: Hdzo5ggPswyv-8ZlW0PVLw
Pred: [(u'gelato', 103), (u'ice cream', 23), (u'chocolate', 12), (u'peanut butter', 9), (u'chocolate chili', 9), (u'pistachio', 8), (u'waffle cone', 8), (u'fruity', 7), (u'mango', 7), (u'strawberry cheesecake', 5)]

True: [(u'peanut butter', 9), (u'ferrero rocher', 8), (u'mango', 8), (u'pistachio', 7), (u'strawberry cheesecake', 6), (u'chocolate', 6), (u'grapefruit', 5), (u'grasshopper', 5), (u'lemon', 5), (u'chocolate chili', 5)]


res_id: dusNIzdCaH6EoLl2hRy6cQ
Pred: [(u'chipotle', 53), (u'burrito', 27), (u'rice', 15), (u'meat', 9), (u'chicken', 7), (u'beans', 7), (u'burrito bowl', 7), (u'lettuce', 6), (u'steak', 5), (u'guacamole', 4)]

True: [(u'burritos', 4), (u'burrito bowl', 3), (u'steak burrito', 2), (u'pork', 1), (u'guacamole', 1), (u'carnitas burrito', 1), (u'chicken salad with guacamole', 1), (u'steaks', 1), (u'green tomatillo salsa', 1), (u'chicken bowl with brown rice', 1)]


res_id: RGK23CEkDfYHWtUbRhA-bQ
Pred: [(u'burger', 14), (u'wings', 1

In [52]:
# precision and recall for model-1
m1_recall = 0.0
m1_precision = 0.0
i=0
for res_id in pred_entities:
    true_y = [ent for ent,_ in true_entities[res_id][:10]]
    pred_y = [ent for ent,_ in pred_entities[res_id][:10]]
    recall, precision = custom_recall(true_y, pred_y)
    m1_recall += recall
    m1_precision += precision
    i += 1
m1_recall /= i
m1_precision /= i
print 'Model-1: Recall: %f, Precision: %f' % (m1_recall, m1_precision)

Model-1: Recall: 0.742857, Precision: 0.657143


### Model 2: based on #occurences of extracted entities in positive reviews (rating >= 4)

In [53]:
# read predicted tags
def get_m2_pred_tags(rev_res_map, pred_file):
    item_scores_pred = {}
    with open(pred_file, 'r') as json_file:
        for line in json_file:
            item_scores = {}
            json_line = json.loads(line)
            review_id = json_line['review_id']
            business_id = rev_res_map[review_id]['business_id']
            if business_id in item_scores_pred:
              item_scores = item_scores_pred[business_id]
            # only for reviews with rating >=4
            if rev_res_map[review_id]['stars'] >= 4.0:
                for entity in json_line['entities']:
                  food_item = entity.lower()
                  if food_item not in item_scores:
                    item_scores[food_item] = 0
                  item_scores[food_item] += 1
            item_scores_pred[business_id] = item_scores
        for res in item_scores_pred:
          item_scores_pred[res] = sorted(item_scores_pred[res].items(), key=lambda x: x[1], reverse=True)
    return item_scores_pred
    

In [54]:
rev_res_map = get_review_restaurant_map()

pred_entities = get_m2_pred_tags(rev_res_map, 'data/filtered_predicted_tags.json')
pred_entities = merge_entities(pred_entities)
pred_entities = sort_entities_map(pred_entities)

true_entities = get_true_entities(rev_res_map)
true_entities = merge_entities(true_entities)
true_entities = sort_entities_map(true_entities)

for res_id in pred_entities:
    print '\nres_id: %s' % (res_id)
    print 'Pred: %s\n' % (pred_entities[res_id][:10])
    print 'True: %s\n' % (true_entities[res_id][:10])


res_id: Hdzo5ggPswyv-8ZlW0PVLw
Pred: [(u'gelato', 72), (u'ice cream', 14), (u'chocolate', 11), (u'pistachio', 7), (u'peanut butter', 6), (u'fruity', 6), (u'chocolate chili', 6), (u'grasshopper', 5), (u'mango', 5), (u'strawberry', 5)]

True: [(u'peanut butter', 9), (u'ferrero rocher', 8), (u'mango', 8), (u'pistachio', 7), (u'strawberry cheesecake', 6), (u'chocolate', 6), (u'grapefruit', 5), (u'grasshopper', 5), (u'lemon', 5), (u'chocolate chili', 5)]


res_id: dusNIzdCaH6EoLl2hRy6cQ
Pred: [(u'chipotle', 30), (u'burrito', 15), (u'rice', 6), (u'steak', 4), (u'lettuce', 4), (u'meat', 3), (u'guacamole', 3), (u'veggie burrito', 3), (u'beans', 3), (u'burrito bowl', 2)]

True: [(u'burritos', 4), (u'burrito bowl', 3), (u'steak burrito', 2), (u'pork', 1), (u'guacamole', 1), (u'carnitas burrito', 1), (u'chicken salad with guacamole', 1), (u'steaks', 1), (u'green tomatillo salsa', 1), (u'chicken bowl with brown rice', 1)]


res_id: RGK23CEkDfYHWtUbRhA-bQ
Pred: [(u'wings', 10), (u'shrimp burger', 

In [55]:
# precision and recall for model-2
m2_recall = 0.0
m2_precision = 0.0
i=0
for res_id in pred_entities:
    true_y = [ent for ent,_ in true_entities[res_id][:10]]
    pred_y = [ent for ent,_ in pred_entities[res_id][:10]]
    recall, precision = custom_recall(true_y, pred_y)
    m2_recall += recall
    m2_precision += precision
    i += 1
m2_recall /= i
m2_precision /= i
print 'Model-2: Recall: %f, Precision: %f' % (m2_recall, m2_precision)

Model-2: Recall: 0.742857, Precision: 0.657143


### Model-3: Compute Popular, by penalizing criticized entities
Rewarding entity from positive review (rating >= 4), as well as penalizing entity from negative review (rating <=3)

In [73]:
# read predicted tags
def get_m3_pred_tags(rev_res_map, pred_file):
    item_scores_pred = {}
    with open(pred_file, 'r') as json_file:
        for line in json_file:
            item_scores = {}
            json_line = json.loads(line)
            review_id = json_line['review_id']
            business_id = rev_res_map[review_id]['business_id']
            if business_id in item_scores_pred:
              item_scores = item_scores_pred[business_id]
            # only for reviews with rating >=4
            
            for entity in json_line['entities']:
                food_item = entity.lower()
                if food_item not in item_scores:
                    item_scores[food_item] = 0
                # rewarding entity from positive review, as well as 
                # penalizing entity from negative review 
                item_scores[food_item] += (1 if rev_res_map[review_id]['stars'] >= 4.0 else -1)
            item_scores_pred[business_id] = item_scores
        for res in item_scores_pred:
          item_scores_pred[res] = sorted(item_scores_pred[res].items(), key=lambda x: x[1], reverse=True)
    return item_scores_pred

In [57]:
rev_res_map = get_review_restaurant_map()

pred_entities = get_m3_pred_tags(rev_res_map, 'data/filtered_predicted_tags.json')
pred_entities = merge_entities(pred_entities)
pred_entities = sort_entities_map(pred_entities)

true_entities = get_true_entities(rev_res_map)
true_entities = merge_entities(true_entities)
true_entities = sort_entities_map(true_entities)

for res_id in pred_entities:
    print '\nres_id: %s' % (res_id)
    print 'Pred: %s\n' % (pred_entities[res_id][:10])
    print 'True: %s\n' % (true_entities[res_id][:10])


res_id: Hdzo5ggPswyv-8ZlW0PVLw
Pred: [(u'gelato', 41), (u'chocolate', 10), (u'pistachio', 6), (u'ice cream', 5), (u'grasshopper', 5), (u'fruity', 5), (u'strawberry', 5), (u'grapefruit', 4), (u'fudge chocolate brownie', 4), (u'guava', 3)]

True: [(u'peanut butter', 9), (u'ferrero rocher', 8), (u'mango', 8), (u'pistachio', 7), (u'strawberry cheesecake', 6), (u'chocolate', 6), (u'grapefruit', 5), (u'grasshopper', 5), (u'lemon', 5), (u'chocolate chili', 5)]


res_id: dusNIzdCaH6EoLl2hRy6cQ
Pred: [(u'chipotle', 7), (u'steak', 3), (u'veggie burrito', 3), (u'burritos', 3), (u'chicken bowl', 2), (u'guacamole', 2), (u'lettuce', 2), (u'brown rice', 2), (u'barbacoa', 2), (u'bottled water', 2)]

True: [(u'burritos', 4), (u'burrito bowl', 3), (u'steak burrito', 2), (u'pork', 1), (u'guacamole', 1), (u'carnitas burrito', 1), (u'chicken salad with guacamole', 1), (u'steaks', 1), (u'green tomatillo salsa', 1), (u'chicken bowl with brown rice', 1)]


res_id: RGK23CEkDfYHWtUbRhA-bQ
Pred: [(u'salmon', 9)

In [58]:
# precision and recall for model-3
m3_recall = 0.0
m3_precision = 0.0
i=0
for res_id in pred_entities:
    true_y = [ent for ent,_ in true_entities[res_id][:10]]
    pred_y = [ent for ent,_ in pred_entities[res_id][:10]]
    recall, precision = custom_recall(true_y, pred_y)
    m3_recall += recall
    m3_precision += precision
    i += 1
m3_recall /= i
m3_precision /= i
print 'Model-3: Recall: %f, Precision: %f' % (m3_recall, m3_precision)

Model-3: Recall: 0.685714, Precision: 0.585714


### Model-4: Differentiated rewards and penalties for entities based on review rating!
Reward entities as below: <br> rating: 5, reward: 2 <br> rating: 4, reward: 1 <br> **rating: 3, reward: 0** <br> rating: 2, reward: -1 <br> rating: 1, reward: -2

In [59]:
# read predicted tags
def get_m4_pred_tags(rev_res_map, pred_file):
    item_scores_pred = {}
    with open(pred_file, 'r') as json_file:
        for line in json_file:
            item_scores = {}
            json_line = json.loads(line)
            review_id = json_line['review_id']
            business_id = rev_res_map[review_id]['business_id']
            if business_id in item_scores_pred:
              item_scores = item_scores_pred[business_id]
            # only for reviews with rating >=4
            
            for entity in json_line['entities']:
                food_item = entity.lower()
                if food_item not in item_scores:
                    item_scores[food_item] = 0
                # differentiated rewards as per review rating
                item_scores[food_item] += (rev_res_map[review_id]['stars'] - 3.0)
            item_scores_pred[business_id] = item_scores
        for res in item_scores_pred:
          item_scores_pred[res] = sorted(item_scores_pred[res].items(), key=lambda x: x[1], reverse=True)
    return item_scores_pred

In [60]:
rev_res_map = get_review_restaurant_map()

pred_entities = get_m4_pred_tags(rev_res_map, 'data/filtered_predicted_tags.json')
pred_entities = merge_entities(pred_entities)
pred_entities = sort_entities_map(pred_entities)

true_entities = get_true_entities(rev_res_map)
true_entities = merge_entities(true_entities)
true_entities = sort_entities_map(true_entities)

for res_id in pred_entities:
    print '\nres_id: %s' % (res_id)
    print 'Pred: %s\n' % (pred_entities[res_id][:10])
    print 'True: %s\n' % (true_entities[res_id][:10])


res_id: Hdzo5ggPswyv-8ZlW0PVLw
Pred: [(u'gelato', 86.0), (u'chocolate', 16.0), (u'ice cream', 13.0), (u'fruity', 9.0), (u'strawberry', 9.0), (u'pistachio', 8.0), (u'grasshopper', 8.0), (u'mango', 8.0), (u'chocolate chili', 8.0), (u'strawberry cheesecake', 7.0)]

True: [(u'peanut butter', 9), (u'ferrero rocher', 8), (u'mango', 8), (u'pistachio', 7), (u'strawberry cheesecake', 6), (u'chocolate', 6), (u'grapefruit', 5), (u'grasshopper', 5), (u'lemon', 5), (u'chocolate chili', 5)]


res_id: dusNIzdCaH6EoLl2hRy6cQ
Pred: [(u'chipotle', 18.0), (u'burritos', 7.0), (u'lettuce', 4.0), (u'steak', 3.0), (u'bottled water', 3.0), (u'veggie burrito', 3.0), (u'guacamole and chips', 2.0), (u'beef chicken salad', 2.0), (u'margarita', 2.0), (u'fleur de', 2.0)]

True: [(u'burritos', 4), (u'burrito bowl', 3), (u'steak burrito', 2), (u'pork', 1), (u'guacamole', 1), (u'carnitas burrito', 1), (u'chicken salad with guacamole', 1), (u'steaks', 1), (u'green tomatillo salsa', 1), (u'chicken bowl with brown rice'

In [61]:
# precision and recall for model-4
m4_recall = 0.0
m4_precision = 0.0
i=0
for res_id in pred_entities:
    true_y = [ent for ent,_ in true_entities[res_id][:10]]
    pred_y = [ent for ent,_ in pred_entities[res_id][:10]]
    recall, precision = custom_recall(true_y, pred_y)
    m4_recall += recall
    m4_precision += precision
    i += 1
m4_recall /= i
m4_precision /= i
print 'Model-4: Recall: %f, Precision: %f' % (m4_recall, m4_precision)

Model-4: Recall: 0.657143, Precision: 0.585714


### Model-5: Computing Sentiment for each entity

Identifies the sentence that has given entity and computes sentiment for it.

In [ ]:
# predict sentiment for each extracted entity, considering current sentence that the entity is part of.
import json
import nltk
from nltk import tokenize
from textblob import TextBlob
nltk.download('punkt')

entities_dict = {}
with open('filtered_predicted_tags.json') as json_file:
    for line in json_file:
      json_line = json.loads(line)
      entities_dict[json_line["review_id"]] = json_line["entities"]
        
annotate_file = 'pred_sent.json'
annotate_writer = open(annotate_file, 'a+')

with open('reviews.json') as json_file:
  for line in json_file:
      json_line = json.loads(line)
      # print(json_line)
      annotation = {}
      annotation['review_id'] = json_line['review_id']
      annotation['entities'] = []
      # annotation['entities'].append({'entity': entity, 'sentiment': sentiment})
      # annotate_writer.write('%s\n' % (json.dumps(annotation)))
      sentences = tokenize.sent_tokenize(json_line["text"])
      pred_entities = entities_dict[json_line["review_id"]]
      for entity in pred_entities:
        entity = entity.lower()
        score = 0
        count = 0
        for i in range(len(sentences)):
          sentence = sentences[i].lower()
          if entity in sentence:
            if i+1<len(sentences):
              testimonial = TextBlob(sentences[i])
            else:
              testimonial = TextBlob(sentences[i])
            score += testimonial.sentiment.polarity
            count += 1
            # print(sentence)
            # print(entity)
            # print(testimonial.sentiment.polarity)
        if count != 0:
          annotation['entities'].append({'entity': entity, 'sentiment': score*1.0/count})
      annotate_writer.write('%s\n' % (json.dumps(annotation)))      

In [69]:
# read predicted tags
def get_m5_pred_tags(rev_res_map, pred_file, sent_threshold):
    item_scores_pred = {}
    with open(pred_file, 'r') as json_file:
        for line in json_file:
            item_scores = {}
            json_line = json.loads(line)
            review_id = json_line['review_id']
            business_id = rev_res_map[review_id]['business_id']
            if business_id in item_scores_pred:
              item_scores = item_scores_pred[business_id]
            
            # consider sentiment for each entity
            # #sent > sent_threshold => +1
            # #sent < -1 * sent_threshold => -1
            for ent in json_line['entities']:
                food_item = ent['entity'].lower()
                if food_item not in item_scores:
                    item_scores[food_item] = 0
                if ent['sentiment'] >= sent_threshold:
                    item_scores[food_item] += 1
                elif ent['sentiment'] <= -1*sent_threshold:
                    item_scores[food_item] += -1
            item_scores_pred[business_id] = item_scores
        for res in item_scores_pred:
          item_scores_pred[res] = sorted(item_scores_pred[res].items(), key=lambda x: x[1], reverse=True)
    return item_scores_pred

In [71]:
rev_res_map = get_review_restaurant_map()

sent_threshold = [0.0, 0.3, 0.4, 0.5, 0.6, 0.7]
for threshold in sent_threshold:
    pred_entities = get_m5_pred_tags(rev_res_map, 'data/pred_sent.json', threshold)
    pred_entities = merge_entities(pred_entities)
    pred_entities = sort_entities_map(pred_entities)

    true_entities = get_true_entities(rev_res_map)
    true_entities = merge_entities(true_entities)
    true_entities = sort_entities_map(true_entities)

    # for res_id in pred_entities:
    #     print '\nres_id: %s' % (res_id)
    #     print 'Pred: %s\n' % (pred_entities[res_id][:10])
    #     print 'True: %s\n' % (true_entities[res_id][:10])

    m5_recall = 0.0
    m5_precision = 0.0
    i=0
    for res_id in pred_entities:
        true_y = [ent for ent,_ in true_entities[res_id][:10]]
        pred_y = [ent for ent,_ in pred_entities[res_id][:10]]
        recall, precision = custom_recall(true_y, pred_y)
        m5_recall += recall
        m5_precision += precision
        i += 1
    m5_recall /= i
    m5_precision /= i
    print 'Model-5: sent_threshold: %f, Recall: %f, Precision: %f' % (threshold, m5_recall, m5_precision)

Model-5: sent_threshold: 0.000000, Recall: 0.642857, Precision: 0.642857
Model-5: sent_threshold: 0.300000, Recall: 0.571429, Precision: 0.600000
Model-5: sent_threshold: 0.400000, Recall: 0.600000, Precision: 0.571429
Model-5: sent_threshold: 0.500000, Recall: 0.500000, Precision: 0.514286
Model-5: sent_threshold: 0.600000, Recall: 0.428571, Precision: 0.471429
Model-5: sent_threshold: 0.700000, Recall: 0.442857, Precision: 0.500000


#### Model-5.b: 

Capture sentiment from sentence where entity appears and from followup sentence too.

In [ ]:
# predict sentiment for each extracted entity, considering two sentences at a time
import json
import nltk
from nltk import tokenize
from textblob import TextBlob
nltk.download('punkt')

entities_dict = {}
with open('filtered_predicted_tags.json') as json_file:
    for line in json_file:
      json_line = json.loads(line)
      entities_dict[json_line["review_id"]] = json_line["entities"]
        
annotate_file = 'pred_sent.json'
annotate_writer = open(annotate_file, 'a+')

with open('reviews.json') as json_file:
  for line in json_file:
      json_line = json.loads(line)
      # print(json_line)
      annotation = {}
      annotation['review_id'] = json_line['review_id']
      annotation['entities'] = []
      # annotation['entities'].append({'entity': entity, 'sentiment': sentiment})
      # annotate_writer.write('%s\n' % (json.dumps(annotation)))
      sentences = tokenize.sent_tokenize(json_line["text"])
      pred_entities = entities_dict[json_line["review_id"]]
      for entity in pred_entities:
        entity = entity.lower()
        score = 0
        count = 0
        for i in range(len(sentences)):
          sentence = sentences[i].lower()
          if entity in sentence:
            if i+1<len(sentences):
              testimonial = TextBlob(sentences[i]+ sentences[i+1])
            else:
              testimonial = TextBlob(sentences[i])
            score += testimonial.sentiment.polarity
            count += 1
            # print(sentence)
            # print(entity)
            # print(testimonial.sentiment.polarity)
        if count != 0:
          annotation['entities'].append({'entity': entity, 'sentiment': score*1.0/count})
      annotate_writer.write('%s\n' % (json.dumps(annotation)))  

In [72]:
rev_res_map = get_review_restaurant_map()

sent_threshold = [0.0, 0.3, 0.4, 0.5, 0.6, 0.7]
for threshold in sent_threshold:
    pred_entities = get_m5_pred_tags(rev_res_map, 'data/pred_sent_2.json', threshold)
    pred_entities = merge_entities(pred_entities)
    pred_entities = sort_entities_map(pred_entities)

    true_entities = get_true_entities(rev_res_map)
    true_entities = merge_entities(true_entities)
    true_entities = sort_entities_map(true_entities)

    # for res_id in pred_entities:
    #     print '\nres_id: %s' % (res_id)
    #     print 'Pred: %s\n' % (pred_entities[res_id][:10])
    #     print 'True: %s\n' % (true_entities[res_id][:10])

    m5_recall = 0.0
    m5_precision = 0.0
    i=0
    for res_id in pred_entities:
        true_y = [ent for ent,_ in true_entities[res_id][:10]]
        pred_y = [ent for ent,_ in pred_entities[res_id][:10]]
        recall, precision = custom_recall(true_y, pred_y)
        m5_recall += recall
        m5_precision += precision
        i += 1
    m5_recall /= i
    m5_precision /= i
    print 'Model-5.b: sent_threshold: %f, Recall: %f, Precision: %f' % (threshold, m5_recall, m5_precision)

Model-5.b: sent_threshold: 0.000000, Recall: 0.657143, Precision: 0.628571
Model-5.b: sent_threshold: 0.300000, Recall: 0.657143, Precision: 0.671429
Model-5.b: sent_threshold: 0.400000, Recall: 0.600000, Precision: 0.585714
Model-5.b: sent_threshold: 0.500000, Recall: 0.542857, Precision: 0.542857
Model-5.b: sent_threshold: 0.600000, Recall: 0.457143, Precision: 0.442857
Model-5.b: sent_threshold: 0.700000, Recall: 0.414286, Precision: 0.371429


### Model Analysis
It is observed from below table that introducing **sentiment analysis** (Model-5.b) **increased precision**, while **Naive Count approach** (Model-1) gave **higher Recall**.

|Model    | Recall|Precision|
|---------|-------|---------|
|Model-1  | 74.28%|    65.7%|
|Model-2  | 74.28%|    65.7%|
|Model-3  |  68.5%|    58.5%|
|Model-4  |  65.7%|    58.5%|
|Model-5  | 64.28%|   64.28%|
|Model-5.b|  65.7%|    67.1%|